To-do:
- Run connectivity on 5 rest subjects (adjust parallelization as needed)
- Assess reliability (permutation tests if not obvious)
- Run averaging and SRM code
- Figure out task decoding and write task_decoding.py


To think about:
- Searchlights instead of parcels as connectivity targets, or as SRM boundaries
- Distance as SRM penalty instead of parcelwise
- DiFuMo atlas instead of Schaefer?
- Reliability.py: permutation tests?
- Write task_decoding.py

In [7]:
import glob
import numpy as np
import nibabel as nib
from nilearn import datasets
from nilearn.image import resample_img, math_img
from nilearn.maskers import NiftiMasker, NiftiLabelsMasker
from joblib import Parallel, delayed
from math import tanh

In [2]:
from sherlock.sherlock_connectivity import load_data_one_session, get_parcellation, compute_fc_one_session

In [3]:
f = 'data/rest/sub-s03_ses-02_task-rest_run-1_space-MNI_desc-optcomDenoised_bold.nii.gz'
c = 'data/confounds/sub-s03_ses-02_task-rest_run-1_desc-confounds_timeseries.tsv'


parcel_labels, parcel_map, parcel_mask = get_parcellation(schaefer_n_rois = 400, resample_target = nib.load(f)) 
voxel_data, parcel_data = load_data_one_session(f,c, parcel_labels, parcel_map, parcel_mask)

c:\Users\ciyer\miniconda3\envs\lab\Lib\site-packages\joblib\memory.py:349: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  return self.func(*args, **kwargs)


In [4]:
# TO-DO!!!

# 1. make it faster
# 2. exclude ones already done
# 3. run it again

In [16]:
vtest = voxel_data[:, 0:10000]
ptest = parcel_data[:, 0:400]

In [55]:
# test to make sure they're the right size, they look ok, etc.
import glob
files = glob.glob('outputs/connectomes/s*')
for f in files:
    test = np.load(f)
    print(np.unique(np.where(np.isnan(test))[0]))

[]
[40919 40941 40963 40964 42814 42815 42834 42835 42855 42856 42880 42931
 42957 42982 42983 43009 44652 44672 44673 44696 44697 44698 44720 44721
 44722 44742 44743 44763 44764 44784 48227 48239 48253 49879 49891 49902
 51661 51677 51693 51709 51710 53704 53724 53743 56058 56089 56120 58702
 58703 58739 58740 58741 58777 61376 61412 61449 63799 63831 63895 65666
 65683 65730 67433 67463 69677 69678 69712 69748 69749 72291 72363 75014
 75047 77600]
[]
[]
[42957 51709 51727 51745 58702 61376 61412 61413 67387 69642 69643 69748
 75110 75147]
[]
[39109 41114 42815 42835 42856 42880 42931 42957 42982 42983 42984 43009
 43010 43035 43036 44652 44672 44673 44696 44697 44698 44720 44721 44722
 44742 44743 44763 44764 44784 48239 48240 48253 48254 48267 48280 48292
 49868 49924 49936 49950 51693 51694 51709 51710 51711 51727 51745 51761
 51780 53804 54033 61342 63830 67387 67410 69642 69643 69677 69748 69785
 72256 72291 72327 72363 72400 72401 75014 75047 75077]
[]
[]
[42931 42957 43009 430

In [56]:
test = np.load('outputs/connectomes\\sub-s03_ses-02_connectome.npy')
idx = np.unique(np.where(np.isnan(test))[0]) # these are the voxels that we struggled with. why?
idx

array([40919, 40941, 40963, 40964, 42814, 42815, 42834, 42835, 42855,
       42856, 42880, 42931, 42957, 42982, 42983, 43009, 44652, 44672,
       44673, 44696, 44697, 44698, 44720, 44721, 44722, 44742, 44743,
       44763, 44764, 44784, 48227, 48239, 48253, 49879, 49891, 49902,
       51661, 51677, 51693, 51709, 51710, 53704, 53724, 53743, 56058,
       56089, 56120, 58702, 58703, 58739, 58740, 58741, 58777, 61376,
       61412, 61449, 63799, 63831, 63895, 65666, 65683, 65730, 67433,
       67463, 69677, 69678, 69712, 69748, 69749, 72291, 72363, 75014,
       75047, 77600], dtype=int64)

In [60]:
from connectivity import load_data_one_session, get_parcellation, get_gm_mask
f = 'data/rest/sub-s03_ses-02_task-rest_run-1_space-MNI_desc-optcomDenoised_bold.nii.gz'
c = 'data/confounds/sub-s03_ses-02_task-rest_run-1_desc-confounds_timeseries.tsv'

parcel_labels, parcel_map, parcel_mask = get_parcellation(schaefer_n_rois = 400, resample_target = nib.load(f) )

voxel_data, parcel_data = load_data_one_session(f,c, parcel_labels, parcel_map, parcel_mask)

# connectome = compute_fc_one_session(voxel_data, parcel_data, zscore=True)

c:\Users\ciyer\miniconda3\envs\lab\Lib\site-packages\joblib\memory.py:349: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  return self.func(*args, **kwargs)


In [86]:
gm_mask = math_img('img >= 0.5', img=nib.load('data/templates/tpl-MNI152NLin2009cAsym_res-02_label-GM_probseg.nii.gz'))
new_mask = math_img('img1 * img2', img1 = gm_mask, img2 = parcel_mask) # gets the intersection of the two

In [ ]:


masker_args = {
    'standardize': 'zscore_sample', # ?
    'n_jobs': 32,
}

voxel_masker = NiftiMasker(
    mask_img = parcel_mask, # CRUCIAL: need this to be the case so we can know which parcel each voxel belongs to later
    **masker_args
)
voxel_data = voxel_masker.fit_transform(f, confounds = c)